<a href="https://colab.research.google.com/github/RonanD10/Tennis-Match-Prediction/blob/main/match_winner_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

from google.colab import drive, files
drive.mount('/content/drive')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

Mounted at /content/drive


In [ ]:
features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed.csv')

matches = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')

<ipython-input-281-502b90c82a09>:3: DtypeWarning: Columns (9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')


In [ ]:
features = features.drop(['A_hand', 'B_hand'], axis=1)

In [ ]:
# Assign grand slam rounds
slams = matches[matches['tourney_level'] == 'G']
slams = slams[(slams['winner_rank'].isna() == False) & (slams['loser_rank'].isna() == False)]
slams.reset_index(drop=True, inplace=True)

f = features
f['slam_round'] = 0

for i in range(len(slams)):
    w_elo = slams.loc[i, 'winner_elo_before']
    l_elo = slams.loc[i, 'loser_elo_before']
    w_rank = slams.loc[i, 'winner_rank']
    l_rank = slams.loc[i, 'loser_rank']
    round = slams.loc[i, 'round']
    index = f[(f['A_elo'] == w_elo) & (f['B_elo'] == l_elo) & (f['A_rank'] == w_rank) & (f['B_rank'] == l_rank)].index[0]
    f.loc[index, 'slam_round'] = round

In [ ]:
# Randomly assign winner and loser to player A and B, whilst maintaining attributes
features['A_won'] = 0

for i in range(len(features)):
    playerA_won = np.random.randint(0, 2)
    features.loc[i, 'A_won'] = playerA_won
    if playerA_won == 0:
        features.loc[i, 'A_rank'], features.loc[i, 'B_rank'] = features.loc[i, 'B_rank'], features.loc[i, 'A_rank']
        features.loc[i, 'A_elo'], features.loc[i, 'B_elo'] = features.loc[i, 'B_elo'], features.loc[i, 'A_elo']
        features.loc[i, 'A_elo_surface'], features.loc[i, 'B_elo_surface'] = features.loc[i, 'B_elo_surface'], features.loc[i, 'A_elo_surface']
        features.loc[i, 'A_avged_elo'], features.loc[i, 'B_avged_elo'] = features.loc[i, 'B_avged_elo'], features.loc[i, 'A_avged_elo']
        features.loc[i, 'A_wins'], features.loc[i, 'B_wins'] = features.loc[i, 'B_wins'], features.loc[i, 'A_wins']
        features.loc[i, 'A_wins_surface'], features.loc[i, 'B_wins_surface'] = features.loc[i, 'B_wins_surface'], features.loc[i, 'A_wins_surface']
        features.loc[i, 'A_12month_form'], features.loc[i, 'B_12month_form'] = features.loc[i, 'B_12month_form'], features.loc[i, 'A_12month_form']
        features.loc[i, 'A_12month_form_surface'], features.loc[i, 'B_12month_form_surface'] =  features.loc[i, 'B_12month_form_surface'], features.loc[i, 'A_12month_form_surface']

In [ ]:
features

,surface,A_rank,B_rank,A_elo_surface,B_elo_surface,A_elo,B_elo,A_avged_elo,B_avged_elo,A_wins,B_wins,A_wins_surface,B_wins_surface,A_12month_form,B_12month_form,A_12month_form_surface,B_12month_form_surface,slam_round,A_won,elo_difference
0,grass,3.0,23.0,1988.523,1644.770,2151.654,1772.521,2070.0885,1708.6455,0,0,0,0,0.833333,0.000000,0.833333,0.000000,1,1,379.133
1,grass,22.0,30.0,1584.360,1501.014,1859.956,1714.681,1722.1580,1607.8475,0,0,0,0,0.666667,0.000000,0.666667,0.000000,1,1,145.275
2,grass,55.0,58.0,1554.535,1792.652,1712.486,1821.925,1633.5105,1807.2885,0,0,0,0,0.500000,0.000000,0.500000,0.000000,1,1,-109.439
3,grass,39.0,15.0,1542.475,1740.677,1794.640,1885.650,1668.5575,1813.1635,0,0,0,0,0.000000,0.750000,0.000000,0.750000,1,0,-91.010
4,grass,97.0,18.0,1503.159,1766.935,1607.397,1922.537,1555.2780,1844.7360,0,0,0,0,0.000000,0.666667,0.000000,0.666667,1,0,-315.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144634,clay,7.0,56.0,1901.149,1590.551,1926.081,1684.596,1913.6150,1637.5735,0,0,0,0,0.717949,0.515152,0.783784,0.533333,0,1,241.485
144635,clay,29.0,47.0,1694.263,1672.773,1724.848,1698.955,1709.5555,1685.8640,0,0,0,0,0.525424,0.680000,0.617647,0.708333,0,1,25.893
144636,clay,19.0,117.0,1554.270,1542.649,1797.649,1567.658,1675.9595,1555.1535,1,0,0,0,0.603448,0.700000,0.500000,0.833333,0,0,229.991
144637,clay,7.0,44.0,1905.735,1605.691,1932.462,1796.400,1919.0985,1701.0455,1,0,0,0,0.717949,0.636364,0.783784,0.666667,0,1,136.062


In [ ]:
features['elo_difference'] = abs(features['A_elo'] - features['B_elo'])
features = pd.get_dummies(features, columns=['surface'])
features.to_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_random.csv', index=False)

In [ ]:
features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_random.csv')

In [ ]:
X = features.drop(['A_won', 'slam_round'], axis=1)
print(len(X))
y = features['A_won']

X_train = X[:int(0.8 * len(X))]
y_train = y[:int(0.8 * len(y))]

test = features[int(0.8 * len(y)):]
test_slams = test[test['slam_round'] > 0]
X_test = test_slams.drop(['A_won', 'slam_round'], axis=1)
y_test = test_slams['A_won']

144639


(5099, 115711)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# lreg = LogisticRegression()
# rfc = RandomForestClassifier()

# lreg.fit(X_train, y_train)
# y_pred = lreg.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Logistic regression accuracy:", accuracy)

# rfc.fit(X_train, y_train)
# y_pred = rfc.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Random forest accuracy:", accuracy)

# compare model coeffs
# gridsearchCV

## Deep Learning Model

In [ ]:
class Model(nn.Module):
  def __init__(self, in_features=21, h1=24, h2=24, out_features=2):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

model = Model()

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(list(y_test))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 1000
losses = []
for i in range(epochs):
  y_pred = model.forward(X_train)
  loss = criterion(y_pred, y_train)
  if loss < 5:
    losses.append(loss.detach().numpy())

  if i % 50 == 0:
    print(f'Epoch: {i} and loss: {loss}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epoch: 0 and loss: 0.41927477717399597
Epoch: 50 and loss: 0.4226013123989105
Epoch: 100 and loss: 0.42005351185798645
Epoch: 150 and loss: 0.41951823234558105
Epoch: 200 and loss: 0.4192602336406708
Epoch: 250 and loss: 0.4191246032714844
Epoch: 300 and loss: 0.41901645064353943
Epoch: 350 and loss: 0.41892099380493164
Epoch: 400 and loss: 0.41883841156959534
Epoch: 450 and loss: 0.4187612235546112
Epoch: 500 and loss: 0.41867920756340027
Epoch: 550 and loss: 0.41860705614089966
Epoch: 600 and loss: 0.4185396432876587
Epoch: 650 and loss: 0.4184684157371521
Epoch: 700 and loss: 0.41840142011642456
Epoch: 750 and loss: 0.4183393716812134
Epoch: 800 and loss: 0.41828030347824097
Epoch: 850 and loss: 0.4182570278644562
Epoch: 900 and loss: 0.41818875074386597
Epoch: 950 and loss: 0.41809308528900146


In [ ]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy_score(y_test, torch.argmax(y_eval, dim=1))}')
print(sum(torch.argmax(y_eval, dim=1) != y_test))
print(len(y_test))

Loss: 0.40476226806640625
Accuracy: 0.8095705040203962
tensor(971)
5099


In [ ]:
# Accuracy per slam round

In [ ]:
plt.plot(range(len(losses)), losses)
plt.ylabel("loss/error")
plt.xlabel('Epoch')

In [ ]:
# Predicting outcome based only on ranking
correct = 0
for i in range(len(features)):
    if features.loc[i, 'A_rank'] < features.loc[i, 'B_rank'] and features.loc[i, 'A_won'] == 1:
        correct += 1
    elif features.loc[i, 'A_rank'] > features.loc[i, 'B_rank'] and features.loc[i, 'A_won'] == 0:
        correct += 1
    else:
        pass

print('Rank accuracy: ', correct/len(features))

# Predicting outcome based only on Elo
correct = 0
for i in range(len(features)):
    if features.loc[i, 'A_elo'] > features.loc[i, 'B_elo'] and features.loc[i, 'A_won'] == 1:
        correct += 1
    elif features.loc[i, 'A_elo'] < features.loc[i, 'B_elo'] and features.loc[i, 'A_won'] == 0:
        correct += 1
    else:
        pass

print('Elo accuracy: ', correct/len(features))

# Predicting outcome based only on 12 month form
correct = 0
for i in range(len(features)):
    if features.loc[i, 'A_12month_form'] > features.loc[i, 'B_12month_form'] and features.loc[i, 'A_won'] == 1:
        correct += 1
    elif features.loc[i, 'A_12month_form'] < features.loc[i, 'B_12month_form'] and features.loc[i, 'A_won'] == 0:
        correct += 1
    else:
        pass

print('12-month form accuracy: ', correct/len(features))

Rank accuracy:  0.6595385753496636
Elo accuracy:  0.6650834145700676
12-month form accuracy:  0.718582125152967
